In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
import tensorflow as tf

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

warnings.filterwarnings("ignore")

In [ ]:
def evaluate_model(model, x, y):
    y_pred_prob = model.predict_proba(x)[:, 1]
    auc_roc = roc_auc_score(y, y_pred_prob)
    return {'auc_roc_curve' : auc_roc}

In [ ]:
seed = 47

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/train.csv', sep=',')

# XGBoost Baseline

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

In [ ]:
params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

# Logistic Regression Baseline

In [ ]:
model = LogisticRegression(random_state=seed, solver='liblinear')
model.fit(x_train, y_train)
results = evaluate_model(model, x_test, y_test)
print(results)

# Feature Engineering

Here wee will experiment creating synthetic features using central tendency statistics.

In [ ]:
geomean = lambda x, axis : np.exp(np.mean(np.log(x), axis=axis))
harmonic_mean = lambda x, axis : len(x) / np.sum(1.0/x, axis=axis) 

funcs = {'mean' : np.mean, 
         'std' : np.std, 
         'var' : np.var, 
         'geo_mean' : geomean, 
         'harmonic_mean' : harmonic_mean, 
         'median' : np.median}

<h3>Feature Engineering - XGBoost</h3>

In [ ]:
results, names = list(), list()

params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

for key in funcs.keys():
    x_train = train_df.drop(['id', 'target'], axis=1)
    x_train[key] = funcs[key](x_train, axis=1)
    y_train = train_df['target']
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)       
    model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])
    
for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

<h3>Feature Engineering - Logistic Regression</h3>

In [ ]:
results, names = list(), list()

for key in funcs.keys():
    x_train = train_df.drop(['id', 'target'], axis=1)
    x_train[key] = funcs[key](x_train, axis=1)
    y_train = train_df['target']
    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)       
    model = LogisticRegression(random_state=seed, solver='liblinear')
    
    model.fit(x_train, y_train)
    result = evaluate_model(model, x_test, y_test)
    names.append(key)
    results.append(result['auc_roc_curve'])

for name, score in zip(names, results):
    print('>%s: %f' % (name, score))

# Grid Search Hyperparameters

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values

<h3>Grid Search Hyperparameters - XGBoost<h3>

In [ ]:
params = {'n_estimators' : [10000, 11000],
          'max_depth' : [2, 3],
          'subsample' : [0.8, 0.9, 1.0],
          'eta' : [0.11, 0.12],
          'colsample_bytree' : [0.3, 0.4],
          'min_child_weight': [5],
          'gamma': [5]
         }

model = XGBClassifier(random_state=seed, tree_method='gpu_hist', predictor='gpu_predictor', use_label_encoder=False, verbosity=0)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
search = GridSearchCV(model, param_grid=params, scoring='roc_auc', refit='roc_auc', n_jobs=-1, cv=cv)

In [ ]:
result = search.fit(x_train, y_train)
print('Best Score: %s' % result.best_score_)
print('Best Hyperparameters: %s' % result.best_params_)

# Ensemble Models

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

<h4>Fitting XGBoost</h4>

In [ ]:
params = {'colsample_bytree': 0.1,
          'eta': 0.12,
          'gamma': 5, 
          'max_depth': 2,
          'min_child_weight': 9,
          'n_estimators': 11000, 
          'subsample': 0.9}

xgboost_model = XGBClassifier(**params, 
                      random_state=seed, 
                      tree_method='gpu_hist', 
                      predictor='gpu_predictor',
                      use_label_encoder=False, 
                      verbosity=0)

xgboost_model.fit(x_train, y_train)
results = evaluate_model(xgboost_model, x_test, y_test)
print(results)

<h4>Fitting Logistic Regression</h4>

In [ ]:
logistic_reg_model = LogisticRegression(random_state=seed, solver='liblinear')
logistic_reg_model.fit(x_train, y_train)
results = evaluate_model(logistic_reg_model, x_test, y_test)
print(results)

<h4>Ensembling both models</h4>

In [ ]:
y_pred_prob_xgboost = xgboost_model.predict_proba(x_test)[:, 1]
y_pred_prob_logistic_reg = logistic_reg_model.predict_proba(x_test)[:, 1]
y_pred_prob = np.mean([y_pred_prob_xgboost, y_pred_prob_logistic_reg], axis=0)
auc_roc = roc_auc_score(y_test, y_pred_prob)
print({'auc_roc_curve' : auc_roc})

# Neural Nets

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1).values
y_train = train_df['target'].values 
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size = 0.2, random_state = seed)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=25)

In [ ]:
results = model.evaluate(x_test, y_test)[1]
print(results)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=128, epochs=35)

In [ ]:
results = model.evaluate(x_test, y_test)[1]
print(results)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
model.fit(x_train, y_train, batch_size=1024, epochs=40)

In [ ]:
results = model.evaluate(x_test, y_test)[1]
print(results)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=35)

In [ ]:
results = model.evaluate(x_test, y_test)[1]
print(results)

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='selu'))
model.add(tf.keras.layers.Dense(128, activation='selu'))
model.add(tf.keras.layers.Dense(64, activation='selu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=35)
results = model.evaluate(x_test, y_test)[1]
print(results)

# Submission

In [ ]:
x_train = train_df.drop(['id', 'target'], axis=1)
y_train = train_df['target']

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(256, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(128, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(64, activation='swish'))
model.add(tf.keras.layers.Dropout(0.1))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.AUC(from_logits=True)])
history = model.fit(x_train, y_train, batch_size=128, epochs=25)

In [ ]:
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-nov-2021/test.csv', sep=',')
x_test = test_df.drop(['id'], axis=1)
target = model.predict(x_test).squeeze()
ids = test_df['id'].values
submission = pd.DataFrame({'id' : ids, 'target' : target})

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)